In [12]:
import numpy as np
import pandas as pd
from pandas import DataFrame as df
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.neighbors import KNeighborsClassifier as KNC
from sklearn.decomposition import PCA, NMF
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_validate,GridSearchCV
from sklearn.metrics import roc_auc_score,accuracy_score
from sklearn.neural_network import MLPClassifier as MLP
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_validate,GridSearchCV,RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA, QuadraticDiscriminantAnalysis as QDA
from sklearn.ensemble import RandomForestClassifier as rf
from sklearn.ensemble import ExtraTreesClassifier as et
from xgboost import XGBClassifier as xgb
from sklearn.decomposition import PCA
from sklearn.metrics import roc_curve, auc

In [15]:
train = pd.read_csv("train_12_3.csv")
valid = pd.read_csv("valid_12_3.csv")

train = train.append(valid)
train_X = train.drop(columns = ["TARGET","ID"])
train_Y = train["TARGET"]

scaler = StandardScaler()
train_X = pd.DataFrame(scaler.fit_transform(train_X), columns=train_X.columns)
# print(train.shape)

test = pd.read_csv("test_12_3.csv")
test_X = test.drop(columns = ["TARGET","ID"])
test_Y = test["TARGET"]
test_X = pd.DataFrame(scaler.transform(test_X), columns=train_X.columns)

/Users/apple/anaconda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/apple/anaconda/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/Users/apple/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:15: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  from ipykernel import kernelapp as app


In [16]:
#all 149 features
train_X_149=train_X.copy()
test_X_149=train_X.copy()

In [20]:
#only 10 PCs 60% variance
n_comp=10
pca10=PCA(n_components=n_comp)
pca10.fit(train_X)
train_X_pca10=df(pca10.transform(train_X))
test_X_pca10=df(pca10.transform(test_X))

#only 76 PCs 99% variance
n_comp=76
pca76=PCA(n_components=n_comp)
pca76.fit(train_X)
train_X_pca76=df(pca76.transform(train_X))
test_X_pca76=df(pca76.transform(test_X))


In [25]:

extraTree=et(n_estimators=100,min_samples_split=11,n_jobs=-1)
extraTree.fit(train_X,train_Y)
importance={}
for i,imp in enumerate(extraTree.feature_importances_):
    importance[train_X.columns[i]]=imp
features2use=[]
sorted_features=[]
for i,f in enumerate(sorted(importance.items(),key=lambda x:-x[1])):
    sorted_features.append(f[0])
    if i<50:
        features2use.append(f[0])
    
# for f in sorted(importance.items(),key=lambda x:-x[1])[:50]:
#     features2use.append(f[0])
    
train_X_pca10top50=train_X.loc[:,features2use]
test_X_pca10top50=test_X.loc[:,features2use]
for col in train_X_pca10.columns:
    train_X_pca10top50['PCA',col]=train_X_pca10.loc[:,col]
    test_X_pca10top50['PCA',col]=test_X_pca10.loc[:,col]
    
train_X_pca2top10=train_X.loc[:,features2use[:10]]
test_X_pca2top10=test_X.loc[:,features2use[:10]]
for col in train_X_pca10.columns[:2]:
    train_X_pca2top10['PCA',col]=train_X_pca10.loc[:,col]
    test_X_pca2top10['PCA',col]=test_X_pca10.loc[:,col]

In [31]:
len(feature2non_dominate)

134

In [32]:
contri=df(pca10.components_,columns=train_X.columns,index = ['PC1','PC2','PC3','PC4','PC5','PC6','PC7','PC8','PC9','PC10'])
contri_list = [(contri[col]**2).sum() for col in contri.columns]
contri_dict={}
for i,val in enumerate(contri_list):
    contri_dict[contri.columns[i]]=val
contri_dict=sorted(contri_dict.items(),key=lambda x:-x[1])
feature2avoid=[entry[0] for i,entry in enumerate(contri_dict) if i<15]
feature2non_dominate=[i for i in sorted_features if i not in feature2avoid][:50]

train_X_pca10next50=train_X.loc[:,feature2non_dominate]
test_X_pca10next50=test_X.loc[:,feature2non_dominate]
for col in train_X_pca10.columns:
    train_X_pca10next50['PCA',col]=train_X_pca10.loc[:,col]
    test_X_pca10next50['PCA',col]=test_X_pca10.loc[:,col]

In [ ]:
def train_base_model(X,y,testX,testY):
    models2consider = [['DecisionTree', DecisionTreeClassifier(), {'max_depth':[3,5,7,9,11,13,15,17,19,21],'min_samples_leaf':[10,20,30,40,100,200,500,1000,2000]}],
                       ['logistic',LogisticRegression(solver='saga', max_iter=2000),{'penalty':['l1','l2'],'C':[0.01,0.1,0.5,1,1.5,2,5,10]}],
                       ['kNN',KNC(),{'n_neighbors':[1,3,5,10,20,50,100,500], 'p': [1,2,3]}],
                       ['LDA', LDA(),{'shrinkage':['auto', 0.01, 0.1,0.3, 0.6], 'solver': ['lsqr', 'eigen']}],
                       ['QDA',QDA(), {'reg_param':[0.001, 0.01, 0.1, 0.3, 0.6]}],
                       ['SVM',SVC(probability=True), {'C':[0.01,0.1,1,5,10,100,1000]}],
                       ['MLP',MLP(), {'activation':['tanh', 'relu'], 'hidden_layer_sizes': [8, 16, 32, [32,16], [16, 8],[16,8,4]], 'learning_rate_init':[0.001,0.01,0.1]}],
                       ['XGB',xgb(), {'max_depth': [3,9,15,21], 'n_estimators':[10,30,50,100,300], 'learning_rate':[0.001,0.01,0.1]}],
                       ['RF', rf(), {'n_jobs':[-1],'n_estimators': [10, 50, 100, 200, 500], 'max_depth': [3, 5, 7, 9, 11, 13, 15, 17, 19, 21], 'min_samples_leaf': [10, 20, 30, 40, 100, 200, 500, 1000, 2000]}]]

    allmodel_resu = []
    for i in range(len(models2consider)):

        model_name = models2consider[i][0]
        clf = models2consider[i][1]
        parameters = models2consider[i][2]

        print(model_name)

        model_resu = {'name':model_name}
        CVresu = GridSearchCV(clf,parameters,cv = 3,scoring='roc_auc',return_train_score=False)
        CVresu = CVresu.fit(X, y)
        best = CVresu.best_estimator_ 
        model_resu['best_param'] = CVresu.best_params_
        model_resu['best_CV_AUC'] = CVresu.best_score_
        model_resu['test_AUC'] = roc_auc_score(test_y ,best.predict_proba(testX)[:,1])
        print(model_resu)
        allmodel_resu.append(model_resu)
    return allmodel_resu

In [ ]:
result_149=train_base_model(train_X_149,train_Y,test_X_149,test_Y)
result_pca10=train_base_model(train_X_pca10,train_Y,test_X_pca10,test_Y)
result_pca76=train_base_model(train_X_pca76,train_Y,test_X_pca76,test_Y)
result_pca10top50=train_base_model(train_X_pca10top50,train_Y,test_X_pca10top50,test_Y)
result_pca2top10=train_base_model(train_X_pca2top10,train_Y,test_X_pca2top10,test_Y)
result_pca10next50=train_base_model(train_X_pca10next50,train_Y,test_X_pca10next50,test_Y)